In [1]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [3]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

In [4]:
iris = dataset("datasets","iris")

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [5]:
X = Matrix(iris[:, 1:4])
irislabels = iris[:,5]

150-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [6]:
X

150×4 Matrix{Float64}:
 5.1  3.5  1.4  0.2
 4.9  3.0  1.4  0.2
 4.7  3.2  1.3  0.2
 4.6  3.1  1.5  0.2
 5.0  3.6  1.4  0.2
 5.4  3.9  1.7  0.4
 4.6  3.4  1.4  0.3
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.9  3.1  1.5  0.1
 5.4  3.7  1.5  0.2
 4.8  3.4  1.6  0.2
 4.8  3.0  1.4  0.1
 ⋮              
 6.0  3.0  4.8  1.8
 6.9  3.1  5.4  2.1
 6.7  3.1  5.6  2.4
 6.9  3.1  5.1  2.3
 5.8  2.7  5.1  1.9
 6.8  3.2  5.9  2.3
 6.7  3.3  5.7  2.5
 6.7  3.0  5.2  2.3
 6.3  2.5  5.0  1.9
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3
 5.9  3.0  5.1  1.8

In [7]:
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)

150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [8]:
function perclass_splits(y, at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids,at)
        push!(keepids, rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [9]:
?randsubseq

search: randsubseq randsubseq! RandomSub StratifiedRandomSub



```
randsubseq([rng=GLOBAL_RNG,] A, p) -> Vector
```

Return a vector consisting of a random subsequence of the given array `A`, where each element of `A` is included (in order) with independent probability `p`. (Complexity is linear in `p*length(A)`, so this function is efficient even if `p` is small and `A` is large.) Technically, this process is known as "Bernoulli sampling" of `A`.

# Examples

```jldoctest
julia> rng = MersenneTwister(1234);

julia> randsubseq(rng, 1:8, 0.3)
2-element Vector{Int64}:
 7
 8
```


In [11]:
trainids = perclass_splits(y, 0.7)
testids = setdiff(1:length(y), trainids)

48-element Vector{Int64}:
   2
   5
   7
  10
  21
  23
  25
  28
  29
  32
  34
  35
  36
   ⋮
 119
 121
 122
 126
 135
 136
 139
 143
 146
 147
 148
 149

In [12]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

In [15]:
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])

Least Squares GLMNet Cross Validation
57 models for 4 predictors in 10 folds
Best λ 0.005 (mean loss 0.053, std 0.006)

In [16]:
# choose the best lambda to predict with
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])
mylambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[trainids,:], y[trainids], lambda = [mylambda])

Least Squares GLMNet Solution Path (1 solutions for 4 predictors in 71 passes):
────────────────────────────
     df   pct_dev          λ
────────────────────────────
[1]   3  0.923115  0.0059989
────────────────────────────

In [21]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)

48×1 Matrix{Float64}:
 0.9828208203589357
 0.9000370580346917
 0.9917157883592901
 0.9212802908937029
 0.9766543668920811
 0.8346734294800593
 1.0093361811693973
 0.9301752588940573
 0.9276316454761064
 1.0721408383811317
 0.8172532957104479
 0.9853644337768865
 0.9225444186402048
 ⋮
 3.282526888414819
 3.0036522404266774
 2.7358617295506904
 2.7322542474267335
 2.49333780966361
 3.096610456422724
 2.56375794920425
 2.71825669466553
 2.949542292174801
 2.729510374968287
 2.7572898635252505
 2.9270349315692883

In [23]:
predictions_lasso = assign_class.(predictions_lasso)
findaccuracy(predictions_lasso, y[testids])

0.9791666666666666

In [27]:
# choose the best lambda to predict with
path = glmnet(X[trainids,:], y[trainids], alpha = 0);
cv = glmnetcv(X[trainids,:], y[trainids], alpha = 0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids], alpha = 0, lambda = [mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path, q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge, y[testids])

1.0

In [28]:
# choose the best lambda to predict with
path = glmnet(X[trainids,:], y[trainids], alpha = 0.5);
cv = glmnetcv(X[trainids,:], y[trainids], alpha = 0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids], alpha = 0.5, lambda = [mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path, q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN, y[testids])

1.0

In [31]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [32]:
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT, y[testids])

0.9583333333333334

In [33]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 6.25
Avg Depth:  4.15

In [34]:
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF, y[testids])

0.9583333333333334

In [35]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

KDTree{StaticArrays.SVector{4, Float64}, Euclidean, Float64}
  Number of points: 102
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true

In [36]:
queries = X[testids,:]

48×4 Matrix{Float64}:
 4.9  3.0  1.4  0.2
 5.0  3.6  1.4  0.2
 4.6  3.4  1.4  0.3
 4.9  3.1  1.5  0.1
 5.4  3.4  1.7  0.2
 4.6  3.6  1.0  0.2
 4.8  3.4  1.9  0.2
 5.2  3.5  1.5  0.2
 5.2  3.4  1.4  0.2
 5.4  3.4  1.5  0.4
 5.5  4.2  1.4  0.2
 4.9  3.1  1.5  0.2
 5.0  3.2  1.2  0.2
 ⋮              
 7.7  2.6  6.9  2.3
 6.9  3.2  5.7  2.3
 5.6  2.8  4.9  2.0
 7.2  3.2  6.0  1.8
 6.1  2.6  5.6  1.4
 7.7  3.0  6.1  2.3
 6.0  3.0  4.8  1.8
 5.8  2.7  5.1  1.9
 6.7  3.0  5.2  2.3
 6.3  2.5  5.0  1.9
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3

In [37]:
idxs, dists = knn(kdtree, queries', 5, true)

([[28, 9, 19, 22, 2], [1, 14, 25, 5, 16], [29, 2, 8, 21, 3], [22, 9, 19, 21, 28], [7, 30, 18, 1, 5], [2, 25, 29, 1, 14], [8, 21, 20, 22, 5], [1, 14, 30, 5, 7], [1, 14, 5, 25, 30], [14, 7, 30, 18, 1]  …  [69, 79, 102, 46, 88], [90, 70, 74, 91, 100], [71, 55, 94, 78, 81], [91, 72, 70, 74, 84], [88, 46, 87, 102, 85], [69, 79, 102, 55, 88], [99, 97, 80, 98, 77], [85, 78, 87, 55, 94], [77, 78, 81, 80, 96], [95, 80, 77, 98, 96]], [[0.14142135623730986, 0.1414213562373099, 0.22360679774997896, 0.24494897427831822, 0.30000000000000016], [0.1414213562373093, 0.17320508075688756, 0.17320508075688767, 0.22360679774997916, 0.26457513110645875], [0.22360679774997871, 0.264575131106459, 0.3000000000000002, 0.31622776601683805, 0.33166247903553986], [0.1732050807568881, 0.17320508075688815, 0.19999999999999993, 0.26457513110645925, 0.2645751311064593], [0.3605551275463991, 0.3741657386773945, 0.4358898943540679, 0.4358898943540679, 0.4472135954999582], [0.5099019513592785, 0.5196152422706636, 0.56568

In [41]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]), 1:size(c,2))
predictions_NN = map(i->parse(Int, string(argmax(possible_labels[i]))), 1:size(c,2))
findaccuracy(predictions_NN, y[testids])

0.9791666666666666

In [43]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

102-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [44]:
model = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Int64, Float64}(36, Int32[4, 15, 17], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [4.3 5.7 … 6.9 5.9; 3.0 4.4 … 3.1 3.0; 1.1 1.5 … 5.1 5.1; 0.1 0.4 … 2.3 1.8], Int32[10, 12, 18, 26, 32, 34, 36, 37, 40, 42  …  84, 85, 87, 88, 90, 92, 94, 96, 99, 102], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 4.3), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 4.5), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 4.9), LIBSVM.SVMNode(1, 5.0), LIBSVM.SVMNode(1, 6.1)  …  LIBSVM.SVMNode(1, 7.7), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.2), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 7.2), LIBSVM.SVMNode(1, 7.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.4), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 5.9)]), 0.0, [0.0 0.015129658050211249; 0.5937486592876166 0.9459546638089537; … ; -0.1702198813731398

In [45]:
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM, y[testids])

0.9791666666666666

In [46]:
overall_accuracies = zeros(7)
methods = ["lasso", "ridge", "EN", "DT", "RF", "kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso, ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge, ytest)
overall_accuracies[3] = findaccuracy(predictions_EN, ytest)
overall_accuracies[4] = findaccuracy(predictions_DT, ytest)
overall_accuracies[5] = findaccuracy(predictions_RF, ytest)
overall_accuracies[6] = findaccuracy(predictions_NN, ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM, ytest)
hcat(methods, overall_accuracies)

7×2 Matrix{Any}:
 "lasso"  0.979167
 "ridge"  1.0
 "EN"     1.0
 "DT"     0.958333
 "RF"     0.958333
 "kNN"    0.979167
 "SVM"    0.979167